In [1]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Conexão database SQLite

consqlite = sqlite3.connect('C:/Users/matheus.candido/sqlite3/exploracao_petroleo_brasil.db', timeout=10)

In [3]:
# Consulta QTD Poços por ano desde 2010

query = f"""
SELECT
        COALESCE(FATO.QTD_POCO, TEMPO.QTD_POCO) QTD_POCO,
        COALESCE(FATO.ANO, TEMPO.ANO) ANO_MES
FROM (
        SELECT
                0 QTD_POCO,
                SUBSTRING(DATA, 1, 7) ANO
	FROM D_TEMPO dt
	WHERE DATA BETWEEN '2010-01-01' AND '2021-06-31'
        --WHERE DATA < '2022-01-01'
	GROUP BY 2
	) TEMPO
LEFT JOIN (
        SELECT 
                COUNT(COD_POCO) QTD_POCO,
                SUBSTRING(DT_INICIO , 1, 7) ANO
	 FROM F_EXPLORACAO fe
         WHERE DT_INICIO BETWEEN '2010-01-01' AND '2021-06-31'
         --WHERE DT_INICIO < '2022-01-01'
	 GROUP BY 2
	 ) FATO
 ON TEMPO.ANO = FATO.ANO
"""

In [4]:
# Importação de QTD Poços por ano

t_pocos = pd.read_sql_query(query, consqlite)
t_pocos.tail(5)

,QTD_POCO,ANO_MES
133,6,2021-02
134,5,2021-03
135,7,2021-04
136,11,2021-05
137,8,2021-06


In [5]:
# Info e conferência de nulos

t_pocos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   QTD_POCO  138 non-null    int64 
 1   ANO_MES   138 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.3+ KB


In [6]:
# Conversão de período para data

t_pocos['ANO_MES'] = pd.to_datetime(t_pocos['ANO_MES'])

In [7]:
# Indexação de data

t_pocos.set_index('ANO_MES', drop=True, inplace=True)

In [8]:
# pacote PyCaret

from pycaret.time_series import *

In [9]:
# Configuração série temporal

setup(t_pocos, fh=12, fold=3)

,Description,Value
0,session_id,2335
1,Target,QTD_POCO
2,Original Data,"(138, 1)"
3,Missing Values,False
4,Approach,Univariate
5,Exogenous Variables,Not Present
6,Transformed Train Target,"(126,)"
7,Transformed Test Target,"(12,)"
8,Transformed Train Exogenous,"(126, 0)"
9,Transformed Test Exogenous,"(12, 0)"


In [10]:
# Criação do modelo

exp_smooth = create_model('exp_smooth')

,cutoff,MAE,RMSE,MAPE,SMAPE,MASE,RMSSE,R2
0,2017-06,4.5576,5.0303,0.2701,0.2574,0.2795,0.2452,0.0419
1,2018-06,3.0093,3.6458,0.2466,0.2281,0.1986,0.1878,-0.3789
2,2019-06,4.0855,4.5769,0.2395,0.2770,0.2888,0.2480,-0.6909
Mean,NaT,3.8841,4.4177,0.2521,0.2541,0.2556,0.2270,-0.3427
SD,NaT,0.6479,0.5763,0.0131,0.0201,0.0405,0.0277,0.3003


In [11]:
# Treino e Teste

plot_model(exp_smooth, plot='train_test_split')

In [12]:
# Finalização do modelo

final = finalize_model(exp_smooth)
final

ExponentialSmoothing(damped_trend=False, damping_trend=None, initial_level=None,
                     initial_seasonal=None, initial_trend=None,
                     initialization_method='estimated', method=None,
                     minimize_kwargs=None, optimized=True, remove_bias=False,
                     seasonal='mul', smoothing_level=None,
                     smoothing_seasonal=None, smoothing_trend=None, sp=12,
                     start_params=None, trend='add', use_boxcox=None,
                     use_brute=True)

In [13]:
# Previsão

plot_model(final, plot='forecast')